In [1]:
"""FIRST MATCHING ALGORITHM in dev"""


#TODO
#exchange action taking
#better __iter__ for stor and for agents in the simulation
#more comments

#finish good matching algo
#put good parameters
#organize the code and create the package
#create data collection
#create new file for automation

'FIRST MATCHING ALGORITHM in dev'

In [2]:
# modules
from system import *
from shape import *


In [3]:
# the algo module (detailed)
class MatchingAlgorithm1:
    """Store all passenger announces
    When a driver look for a passenger, compute and send the list of coherent announces (based on window time)
    When a match is made delete the announce"""
    def __init__(self,benefits_function,simu):
        self.passengerList={}
        self.benefits=benefits
        self.simulation=simu
    def addPassenger(self,agent,lastDeparture,origin,destination):
        """Add a passenger with all needed info"""
        self.passengerList[agent]={"ld":lastDeparture,"O":origin,"D":destination}
    def checkPotentialMatching(self,departureTime,origin):#information of driver
        """send the list of compatible announce (with the departureTime)"""
        out=[]
        for passenger in self.passengerList:
            p_info=self.passengerList[passenger]
            travelTime=self.simulation.network.travel_time(origin,p_info["O"])#travel time from the driver origin to the passenger origin
            if departureTime+travelTime <= p_info["ld"]:#check compatibility
                out.append(self.sendAnnounce(passenger,p_info))
        return out
    def sendAnnounce(self,agent,info):
        """set the information send to a driver when a coherent passenger is found"""
        bene=self.benefits(info["O"],info["D"])
        return (agent,{"O":info["O"],"D":info["D"],"b":bene})
    def retreivePassenger(self,agent):
        """delete an announce when accepted or obsolete"""
        if agent in self.passengerList:
            del(self.passengerList[agent])
            return True
        else:
            return False
        
        
        
        
        

        
        
#EVENT
class PublishAnnounce(Event):
    """Event when a passenger publish an announce"""
    def __init__(self, agent):
        self.time=agent.publishing_time
        self.agent=agent
    def run(self,simulation):
        agent=self.agent
        simulation.matchingAlgo.addPassenger(agent,agent.last_departure_time,agent.position,agent.destination)
        simulation.put(RetreiveAnnounce(agent,agent.last_departure_time))
        self.agent(self.time,"waiting",position=agent.position)
    def __str__(self):
        return super().__str__() + " by agent " + str(self.agent.id_number)
        
class RetreiveAnnounce(Event):
    """Delete a passenger's announce
        if really delete it means the passenger didn't find a proper driver
        otherwise this event is useless"""
    def __init__(self,agent,time):
        self.time=time
        self.agent=agent
    def run(self,simulation):
        if simulation.matchingAlgo.retreivePassenger(self.agent):
            self.agent(self.time,"missed")
    def __str__(self):
        return super().__str__() + " by agent " + str(self.agent.id_number)
        
        
class WatchAnnounce(Event):
    """A driver ask for potential passengers"""
    def __init__(self,agent,watching_time):
        self.time=watching_time
        self.agent=agent
    def __str__(self):
        return super().__str__() + " by agent " + str(self.agent.id_number)
    def run(self,simulation):
        #compute at what time the driver can leave:
        possible_departure=max(self.agent.departure_window[0],self.time)
        #look all potential matchings
        potentialMatching=simulation.matchingAlgo.checkPotentialMatching(possible_departure,self.agent.position)
        agentMatched=None
        bestRate=0
        for match in potentialMatching:
            rate=self.agent.rate(match[1])
            if rate > bestRate:#driver accept the match
                agentMatched=match[0]
        if agentMatched:#we have a match!
            simulation.matchingAlgo.retreivePassenger(agentMatched)
            self.agent(self.time,"matched",passenger=agentMatched.id_number)
            agentMatched(self.time,"matched",driver=self.agent.id_number)
            l_points=[("Od",self.agent.position),("Op",agentMatched.position),("Dp",agentMatched.destination),("Dd",self.agent.destination)]
            l_agents=[[self.agent,"Od","Dd"],[agentMatched,"Op","Dp"]]
            t=Travel(possible_departure,l_points,l_agents)
            simulation.put(t)
        else:
            next_watching=self.time+self.agent.repetition_time
            if next_watching > self.agent.departure_window[1]:#too late the driver leaves
                self.agent(self.time,"alone")
                simulation.put(Travel(possible_departure,[("Od",self.agent.position),("Dd",self.agent.destination)],[[self.agent,"Od","Dd"]]))#simplify?
            else:
                simulation.put(WatchAnnounce(self.agent,next_watching))
                self.agent(self.time,"watching",position=self.agent.position)
        
class Travel(Event):
    """Has a list of points for the trajectory
    Also has the list of travellers with their start and end points"""
    def __init__(self,time,points_list,agents_list):
        """points_list -> list of tuples (point_name,point_coordinates) in the order of the travel
        agent_list -> list of list [agent,origin_name,destination_name]
                                origin_name="" if already travelling """
        self.time=time
        self.points=points_list
        self.agents=agents_list
    def __str__(self):
        out=super().__str__()+" by agents"
        for agent in self.agents:
            out+= " " + str(agent[0].id_number)
        return out
    def run(self,simulation):
        ###first lets take a look at who arrived at destination:
        start_name,start_point=self.points[0]
        finished=[]
        for passenger in self.agents:
            if passenger[1] is "" and passenger[2] is start_name:#he was moving and arrived
                passenger[0](self.time,"arrived",point=start_point)
                finished.append(passenger)
        self.agents=[x for x in self.agents if x not in finished]
        ###then lets watch if the travel is finished
        if len(self.points) is 1:#arrived to the last point -> nothing happens
            assert len(self.agents) is 0#otherwise some people were not stopped
        else:###if not the case lets identify who started and lets built the next event
            for passenger in self.agents:
                if passenger[1] is start_name:#he begins
                    passenger[1]=""
            next_name,next_point=self.points[1]
            for passenger in self.agents:
                if passenger[1] is "":#he is moving
                    passenger[0](self.time,"moving",start=start_point,end=next_point)
            travelled_time=simulation.network.travel_time(start_point,next_point)
            del self.points[0]
            simulation.put(Travel(self.time+travelled_time,self.points,self.agents))
            
            
            
            
            
#AGENTS
class Passenger(Agent):
    """ask for a drive"""
    attributes=["publishing_time","last_departure_time","position","destination"]
    def compute(self,simulation):
        simulation.put(PublishAnnounce(self))
        
class Driver(Agent):
    """propose a drive"""
    attributes=["first_watching_time","repetition_time","departure_window","position","destination","rate"]
    def compute(self,simulation):
        simulation.put(WatchAnnounce(self,self.first_watching_time))



In [26]:
# CREATE A SIMULATION
speed=3
X=(0,200)
Y=(0,300)
R=300
end=3*60
#N=rectangle(X,Y,speed)
N=circle(R,speed)
T=Timer(end)
simu=Simulation(N,T)

#WITH THE ALGORITHM
passengerCostPerTime=6
fixBenefit=3
def benefits(origin,destination):
    return simu.network.travel_time(origin,destination)*passengerCostPerTime+fixBenefit
simu.matchingAlgo=MatchingAlgorithm1(benefits,simu)

# CREATE THE DEMAND
N_driver=60
N_passenger=60
def rater(*args,**kwargs):
    return 1
#GENERATORS
def SimpleDriver(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    return Driver(first_watching_time=t,repetition_time=20,departure_window=(t+100,t+140),position=O,destination=D,rate=rater)
def SimplePassenger(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    return Passenger(publishing_time=t,last_departure_time=t+100,position=O,destination=D)


demand=[]#just a list of agents
for i in range(N_driver):
    demand.append(SimpleDriver(simu))
for i in range(N_passenger):
    demand.append(SimplePassenger(simu))



In [27]:
#here we go
simu.add(demand)
#simu.set_action(print)
simu()

In [23]:
for p in demand:
    print(p)
    
nb_driver_alone=0
for p in demand:
    if isinstance(p,Driver):
        for action in p.story:
            if action[0] is "alone":
                nb_driver_alone+=1
                break
print(nb_driver_alone)

nb_passenger_missed=0
for p in demand:
    if isinstance(p,Passenger):
        for action in p.story:
            if action[0] is "missed":
                nb_passenger_missed+=1
                break
print(nb_passenger_missed)

Agent 1561 of type Driver.
begin at 369.438698244856 to ('arrived', {'point': (-49.119455514891996, -166.77278557743213)}) 
begin at 212.8660121776843 to ('moving', {'end': (-49.119455514891996, -166.77278557743213), 'start': (136.47018162559493, 264.72639262186)}) 
begin at 126.17753127001625 to ('moving', {'end': (136.47018162559493, 264.72639262186), 'start': (-5.042358615397234, 46.53327757637163)}) 
begin at 105.84874122090706 to ('moving', {'end': (-5.042358615397234, 46.53327757637163), 'start': (-6.957680901282572, 107.48956424385193)}) 
begin at 45.84874122090706 to ('matched', {'passenger': 1648}) 
begin at 25.848741220907055 to ('watching', {'position': (-6.957680901282572, 107.48956424385193)}) 
begin at 5.848741220907055 to ('watching', {'position': (-6.957680901282572, 107.48956424385193)}) 

Agent 1562 of type Driver.
begin at 273.6531835409758 to ('arrived', {'point': (-10.2756572189748, 45.243797411453386)}) 
begin at 264.2230802131524 to ('moving', {'end': (-10.275657

In [24]:
#draw :
from drawing import *

def position(agent,t,network):
    action,to=agent.story[t]
    if action[0] is "waiting" or action[0] is "watching" :
        return action[1]["position"]
    if action[0] is "matched" :
        return position(agent,to-1,network)
    if action[0] is "moving":
        D=t-to#time since last point
        D=network.travel_distance(D)#distance made with that time
        drel=(action[1]["end"][0]-action[1]["start"][0],action[1]["end"][1]-action[1]["start"][1])#relative vector
        module=sqrt(drel[0]**2+drel[1]**2)
        dx=D*drel[0]/module#distance made in each direction
        dy=D*drel[1]/module
        return (action[1]["start"][0]+dx,action[1]["start"][1]+dy)


def positionList(agents,network):
    def out_f(t):
        out=[]
        for agent in agents:
            out.append(position(agent,t,network))
        return out
    return out_f


p=positionList(demand,simu.network)
    
def create_objects(ax):
    passenger,= ax.plot([], [],'bo', ms=4)
    driver,= ax.plot([], [],'ro', ms=6)
    return [driver,passenger]

def updateFrom(agents,network):
    def update(objects,t):
        p=[]
        d=[]
        for agent in agents:
            if isinstance(agent,Passenger):
                p.append(position(agent,t,network))
            if isinstance(agent,Driver):
                d.append(position(agent,t,network))
        if p:
            x=[x[0] for x in p if x is not None]
            y=[x[1] for x in p if x is not None]
            objects[1].set_data(x,y)
        if d:
            x=[x[0] for x in d if x is not None]
            y=[x[1] for x in d if x is not None]
            objects[0].set_data(x,y)
        return objects
    return update

update=updateFrom(demand,simu.network)

In [25]:

dr=Drawing_from_simulation(simu,create_objects,update)
dr.set_options(margin=20,real_size=(10,10),time_interval=10,time_coef=30,repetition=True,hide_axes=True)

#delete or change the use of X,Y
#propose several color, for several agent
#make it work
dr()

In [ ]:
help(Agent)